https://www.kaggle.com/huikang/lux-ai-working-title-bot#[Lux-AI]-Working-Title-Bot

1. x, y 좌표를 tuple로 보관
2. typing library를 사용하여 Set, List 등의 타입을 다 지정을 함
3. disjoint set을 사용하여 city를 구분


In [1]:
!pip install kaggle-environments -U
!git clone https://github.com/Lux-AI-Challenge/Lux-Design-2021.git
!cp -r ./Lux-Design-2021/kits/python/simple/lux .

fatal: destination path 'Lux-Design-2021' already exists and is not an empty directory.


<h1>Upgrade Game Kit

In [2]:
%%writefile lux/game_position.py

from lux import game
import random
from typing import List, Set, Tuple

from .constants import Constants

DIRECTIONS = Constants.DIRECTIONS

class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __sub__(self, pos: 'Position') -> int:
        #  Manhattan Distance
        return abs(pos.x - self.x) + abs(pos.y - self.y)

    def distance_to(self, pos: 'Position'):
        return self - pos

    def is_adjacent(self, pos: 'Position') -> bool:
        return (self - pos) <= 1

    def __eq__(self, pos: 'Position') -> bool:
        return self.x == pos.x and self.y == pos.y

    def equals(self, pos: 'Position'):
        return self == pos

    # position이 direction 방향으로 units 만큼 움직인다.
    def translate(self, direction, units) -> 'Position':
        if direction == DIRECTIONS.NORTH:
            return Position(self.x, self.y - units)
        elif direction == DIRECTIONS.EAST:
            return Position(self.x + units, self.y)
        elif direction == DIRECTIONS.SOUTH:
            return Position(self.x, self.y + units)
        elif direction == DIRECTIONS.WEST:
            return Position(self.x - units, self.y)
        elif direction == DIRECTIONS.CENTER:
            return Position(self.x, self.y)

    def __str__(self) -> str:
        return f"({self.x}, {self.y})"

    def __iter__(self):
        for i in (self.x, self.y):
            yield i

Overwriting lux/game_position.py


In [3]:
# test game_postion
from lux.game_position import Position

pos1 = Position(1, 2)
pos2 = Position(2, 4)

print("pos1 - pos2", pos1 - pos2)

print("test Position __iter__()")
for i in pos1:
    print(i)

pos1 - pos2 3
test Position __iter__()
1
2


In [4]:
import time
import heapq
from collections import defaultdict, deque
# type annotation
from typing import DefaultDict, Dict, List, Tuple, Set

import numpy as np

from lux.constants import Constants
from lux.game_map import GameMap, RESOURCE_TYPES
from lux.game_objects import Player, Unit, City, CityTile
from lux.game_position import Position
from lux.game_constants import GAME_CONSTANTS

INPUT_CONSTANTS = Constants.INPUT_CONSTANTS

class Mission:
    def __init__(self, unit_id: str, target_position: Position, target_action: str = ""):
        self.target_position: Postion = target_position
        self.target_action: str = target_action
        self.unit_id: str = unit_id
        self.delays: int = 0

    def __str__(self):
        return " ".join([str(self.target_position), self.target_action])

class Missions(defaultdict):
    def __init__(self):
        self: DefaultDict[str, Mission] = defaultdict(Mission)

    def add(self, mission: Mission):
        self[mission.unit_id] = mission

    # 필요하지 않은 Mission들을 삭제하는 method
    def cleanup(self, player: Player,
                player_city_tile_xy_set: Set[Tuple],
                opponent_city_tile_xy_set: Set[Tuple],
                convolved_collectable_tiles_xy_set: Set[Tuple]):
        
        # 현재 Missions에 저장된 모든 list를 확인
        for unit_id in list(self.keys()):
            mission: Mission = self[unit_id]

            # 현재 player가 사라졌으면 삭제
            if unit_id not in player.units_by_id:
                del self[unit_id]
                continue

            unit : Unit = player.units_by_id[unit_id]

            # target_action이 bcity이면
            if mission.target_action and mission.target_action[:5] == "bcity":
                # 만약 유닛이 가지고 있는 cargo가 하나도 없으면 지워버림
                if unit.cargo == 0:
                    del self[unit_id]
                    continue

                # 상대방 city tile로 가고 있다면 지워버림
                if tuple(mission.target_position) in opponent_city_tile_xy_set:
                    del self[unit_id]
                    continue

                # 만약 현재 unit의 위치가 city안에 있다면 지워버림
                if tuple(unit.pos) in player_city_tile_xy_set:
                    del self[unit_id]
                    continue

                # 만약 resource가 없는 쪽으로 가고 있다면 지워버림
                if tuple(mission.target_position) not in convolved_collectable_tiles_xy_set:
                    del self[unit_id]
                    continue

    def __str__(self):
        return " ".join([unit_id + " " + str(x) for unit_id, x in self.items()])


    def get_targets(self):
        return [mission.target_position for unit_id, mission in self.items()]

    def get_targets_and_actions(self):
        return [(mission.target_position, mission.target_action) for unit_id, mission in self.items()]


class DisjointSet:
    def __init__(self):
        self.parent = {}
        self.sizes = defaultdict(int)
        self.points = defaultdict(int)
        self.num_sets = 0

    def find(self, a, point = 0):
        # 만약 a가 새로운 노드이면
        if a not in self.parent:
            self.parent[a] = a
            self.sizes[a] += 1
            self.points[a] += point
            self.num_sets += 1
        acopy = a

        # a의 root를 탐색
        while a != self.parent[a]:
            a = self.parent[a]

        # 만약 root와 자기 자신이 다르면, root 노드의 하위 노드들의 부모를 모두 a로 바꿈
        while acopy != a:
            # 나의 부모를 a로 만들고, 
            self.parent[acopy], acopy = a, self.parent[acopy]
        return a

    def union(self, a, b):
        a, b = self.find(a), self.find(b)
        
        # 만약 집합이 다른 경우
        if a != b:
            # 큰 쪽을 a로 치환
            if self.sizes[a] < self.sizes[b]:
                a, b = b, a

            self.num_sets -= 1
            self.parent[b] = a
            self.sizes[a] += self.sizes[b]
            self.points[a] += self.points[b]

    def get_size(self, a):
        return self.sizes[self.find(a)]

    def get_point(self, a):
        return self.points[self.find(a)]

    #해당 그룹에 있는 모든 element들을 저장해서 return
    def get_groups(self):
        groups = defaultdict(list)
        for element in self.parent:
            leader = self.find(element)
            if leader:
                groups[leader].append(element)

        return groups

    def get_group_count(self):
        return sum(self.points[leader] > 1 for leader in self.get_groups().keys())


class Game:

    compute_start_time = -1

    def _initialiaze(self, messages):

        self.player_id: int = int(messages[0])
        self.turn: int = -1

        mapInfo = messages[1].split(" ")
        self.map_width: int = int(mapInfo[0])
        self.map_height: int = int(mapInfo[1])
        self.map : GameMap = GameMap(self.map_width, self.map_height)
        self.players: List[Player] = [Player(0), Player(1)]

        self.x_iteration_order = list(range(self.map_width))
        self.y_iteration_order = list(range(self.map_height))

        self.dirs: List = [
            Constants.DIRECTIONS.NORTH,
            Constants.DIRECTIONS.EAST,
            Constants.DIRECTIONS.SOUTH,
            Constants.DIRECTIONS.WEST,
            Constants.DIRECTIONS.CENTER
        ]

        self.dirs_dxdy: List = [(0, -1), (1, 0), (0, 1), (-1, 0), (0, 0)]

    def fix_iteration_order(self):
        assert len(self.player.cities) == 1
        assert len(self.opponent.cities) == 1

        px, py = tuple(list(self.player.cities.values())[0].citytiles[0].pos)
        ox, oy = tuple(list(self.opponent.cities.values())[0].citytiles[0].pos)

        flipping = False

        self.x_order_coefficient = 1
        self.y_order_coefficient = 1

        if px == ox:
            if py < oy:
                flipping = True
                self.y_iteration_order = self.y_iteration_order[::-1]
                self.y_order_coefficient = -1

                idx1, idx2 = 0, 2
        elif py == oy:
            if px < ox:
                flipping = True
                self.x_iteration_order = self.x_iteration_order[::-1]
                self.x_order_coefficient = -1

                idx1, idx2 = 1, 3

        else:
            assert False

        if flipping:
            self.dirs[idx1], self.dirs[idx2] = self.dirs[idx2], self.dirs[idx1]
            self.dirs_dxdy[idx1], self.dirs_dxdy[idx2] = self.dirs_dxdy[idx2], self.dirs_dxdy[idx1]

    def _end_turn(self):
        print("D_FINISH")

    def _reset_player_states(self):
        self.players[0].units = []
        self.players[0].cities = {}
        self.players[0].city_tile_count = 0
        self.players[1].units = []
        self.players[1].cities = {}
        self.players[1].city_tile_count = 0

        self.player: Player = self.players[self.player_id]
        self.opponent: Player = self.players[1 - self.player_id]

    # message에 어떤 update를 할 지 담아서 실행
    def _update(self, messages):
        self.map = GameMap(self.map_width, self.map_height)
        self.turn += 1
        self._reset_player_states()

        for update in messages:
            if update == "D_DONE":
                break
            strs = update.split(" ")
            input_identifier = strs[0]

            if input_identifier == INPUT_CONSTANTS.RESEARCH_POINTS:
                team = int(strs[1])   # probably player_id
                self.players[team].research_points = int(strs[2])

            elif input_identifier == INPUT_CONSTANTS.RESOURCES:
                r_type = strs[1]
                x = int(strs[2])
                y = int(strs[3])
                amt = int(float(strs[4]))
                self.map._setResource(r_type, x, y, amt)

            elif input_identifier == INPUT_CONSTANTS.UNITS:
                unittype = int(strs[1])
                team = int(strs[2])
                unitid = strs[3]
                x = int(strs[4])
                y = int(strs[5])
                cooldown = float(strs[6])
                wood = int(strs[7])
                coal = int(strs[8])
                uranium = int(strs[9])
                unit = Unit(team, unittype, unitid, x, y, cooldown, wood, coal, uranium)
                # units 리스트에 append
                self.players[team].units.append(unit)
                # map에 unit 표기
                self.map.get_cell(x, y).unit = unit

            elif input_identifier == INPUT_CONSTANTS.CITY:
                team = int(strs[1])
                cityid = strs[2]
                fuel = float(strs[3])
                lightupkeep = float(strs[4])
                # city 정보 업데이트
                self.players[team].cities[cityid] = City(team, cityid, fuel, lightupkeep)

            elif input_identifier == INPUT_CONSTANTS.CITY_TILES:
                team = int(strs[1])
                cityid = strs[2]
                x = int(strs[3])
                y = int(strs[4])
                cooldown = float(strs[5])
                # city tile로 city 받아오고
                city = self.players[team].cities[cityid]
                # city tile add
                citytile = city._add_city_tile(x, y, cooldown)
                # map에 표기
                self.map.get_cell(x, y).citytile = citytile
                self.players[team].city_tile_count += 1

            elif input_identifier == INPUT_CONSTANTS.ROADS:
                x = int(strs[1])
                y = int(strs[2])
                road = float(strs[3])
                # map에 표기
                self.map.get_cell(x, y).road = road

        self.player.make_index_units_by_id()
        self.opponent.make_index_units_by_id()


    def calculate_features(self, missions: Missions):
        # 기본적인 상수 정보
        self.wood_fuel_rate = GAME_CONSTANTS["PARAMETERS"]["RESOURCE_TO_FUEL_RATE"][RESOURCE_TYPES.WOOD.upper()]
        self.wood_collection_rate = GAME_CONSTANTS["PARAMETERS"]["WORKER_COLLECTION_RATE"][RESOURCE_TYPES.WOOD.upper()]
        self.coal_fuel_rate = GAME_CONSTANTS["PARAMETERS"]["RESOURCE_TO_FUEL_RATE"][RESOURCE_TYPES.COAL.upper()]
        self.coal_collection_rate = GAME_CONSTANTS["PARAMETERS"]["WORKER_COLLECTION_RATE"][RESOURCE_TYPES.COAL.upper()]
        self.uranium_fuel_rate = GAME_CONSTANTS["PARAMETERS"]["RESOURCE_TO_FUEL_RATE"][RESOURCE_TYPES.URANIUM.upper()]
        self.uranium_collection_rate = GAME_CONSTANTS["PARAMETERS"]["WORKER_COLLECTION_RATE"][RESOURCE_TYPES.URANIUM.upper()]

        # 낮/밤 시간 정보
        self.night_turns_left = (360 - self.turn)//40 * 10 + min(10, (360 - self.turn)%40)

        self.turns_to_night = (30 - self.turn)%40
        self.turns_to_night = 0 if self.turns_to_night > 30 else self.turns_to_night

        self.turns_to_dawn = (40 - self.turn%40)
        self.turns_to_dawn = 0 if self.turns_to_dawn > 10 else self.turns_to_dawn

        self.is_day_time = self.turns_to_dawn == 0

        # 아래서 함수 하나씩 확인
        self.calculate_matrix()
        self.calculate_resource_matrix()
        self.calculate_resource_groups()
        self.calculate_distance_matrix()

        self.repopulate_targets(missions)

        self.heuristics_from_positions: Dict = dict()


    def init_matrix(self, default_value=0):
        return np.full((self.map_height,self.map_width), default_value)


    def calculate_matrix(self):

        # resource 관련 matrix
        self.wood_amount_matrix = self.init_matrix()
        self.coal_amount_matrix = self.init_matrix()
        self.uranium_amount_matrix = self.init_matrix()
        self.all_resource_amount_matrix = self.init_matrix()

        # city tile matrix
        self.player_city_tile_matrix = self.init_matrix()
        self.opponent_city_tile_matrix = self.init_matrix()

        # unit matrix
        self.player_units_matrix = self.init_matrix()
        self.opponent_units_matrix = self.init_matrix()

        self.empty_tile_matrix = self.init_matrix()

        self.buildable_tile_matrix = self.init_matrix()

        # map 전체 탐색
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:

                cell = self.map.get_cell(x, y)

                is_empty = True
                is_buildable = True

                # cell에 unit 표기
                if cell.unit:
                    is_empty = False
                    if cell.unit.team == self.player_id:
                        self.player_units_matrix[y,x] += 1
                    else:   # unit belongs to opponent
                        self.opponent_units_matrix[y,x] += 1

                # cell에 resource 표기
                if cell.has_resource():
                    is_empty = False
                    is_buildable = False
                    if cell.resource.type == RESOURCE_TYPES.WOOD:
                        self.wood_amount_matrix[y,x] += cell.resource.amount
                    if cell.resource.type == RESOURCE_TYPES.COAL:
                        self.coal_amount_matrix[y,x] += cell.resource.amount
                    if cell.resource.type == RESOURCE_TYPES.URANIUM:
                        self.uranium_amount_matrix[y,x] += cell.resource.amount
                    self.all_resource_amount_matrix[y,x] += cell.resource.amount

                # cell에 citytile 표기
                elif cell.citytile:
                    is_empty = False
                    is_buildable = False
                    if cell.citytile.team == self.player_id:
                        self.player_city_tile_matrix[y,x] += 1
                    else:   # city tile belongs to opponent
                        self.opponent_city_tile_matrix[y,x] += 1

                if is_empty:
                    self.empty_tile_matrix[y,x] += 1

                if is_buildable:
                    self.buildable_tile_matrix[y,x] += 1

        # binary matrix를 따로 만드네...
        self.wood_exist_matrix = (self.wood_amount_matrix > 0).astype(int)
        self.coal_exist_matrix = (self.coal_amount_matrix > 0).astype(int)
        self.uranium_exist_matrix = (self.uranium_amount_matrix > 0).astype(int)
        self.all_resource_exist_matrix = (self.all_resource_amount_matrix > 0).astype(int)

        # 주변에 wood, coal, uranium이 몇개나 있는지 저장
        self.wood_side_matrix = self.convolve(self.wood_exist_matrix) * self.empty_tile_matrix
        self.coal_side_matrix = self.convolve(self.coal_exist_matrix) * self.empty_tile_matrix
        self.uranium_side_matrix = self.convolve(self.uranium_exist_matrix) * self.empty_tile_matrix

        self.convert_into_sets()

    # matrix가 들어오면 안에 값이 있는 cell을 set object로 변환
    def populate_set(self, matrix, set_object):
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if matrix[y,x] > 0:
                    set_object.add((x,y))


    def convert_into_sets(self):
        self.wood_exist_xy_set = set()
        self.coal_exist_xy_set = set()
        self.uranium_exist_xy_set = set()
        self.player_city_tile_xy_set = set()
        self.opponent_city_tile_xy_set = set()
        self.player_units_xy_set = set()
        self.opponent_units_xy_set = set()
        self.empty_tile_xy_set = set()
        self.buildable_tile_xy_set = set()

        # 지금까지 만든 matrix 다 set으로 변환
        for set_object, matrix in [
            [self.wood_exist_xy_set,            self.wood_exist_matrix],
            [self.coal_exist_xy_set,            self.coal_exist_matrix],
            [self.uranium_exist_xy_set,         self.uranium_exist_matrix],
            [self.player_city_tile_xy_set,      self.player_city_tile_matrix],
            [self.opponent_city_tile_xy_set,    self.opponent_city_tile_matrix],
            [self.player_units_xy_set,          self.player_units_matrix],
            [self.opponent_units_xy_set,        self.opponent_units_matrix],
            [self.empty_tile_xy_set,            self.empty_tile_matrix],
            [self.buildable_tile_xy_set,        self.buildable_tile_matrix]]:

            self.populate_set(matrix, set_object)


        # 가장자리 set 만들기
        self.xy_out_of_map: Set = set()
        
        for y in [-1, self.map_height]:
            for x in range(self.map_width):
                self.xy_out_of_map.add((x,y))

        for y in range(self.map_height):
            for x in [-1, self.map_width]:
                self.xy_out_of_map.add((x,y))


        # 가지 못하는 곳에 대한 set을 만듦
        self.occupied_xy_set = (self.player_units_xy_set | self.opponent_units_xy_set | \
                                self.opponent_city_tile_xy_set | self.xy_out_of_map) \
                                - self.player_city_tile_xy_set


    def calculate_distance_matrix(self, blockade_multiplier_value=100):

        # 일단 map_height + map_width로 matrix를 채워둠
        self.distance_from_edge = self.init_matrix(self.map_height + self.map_width)

        # 가장자리로 부터 거리를 matrix로 만듦
        for y in range(self.map_height):
            y_distance_from_edge = min(y, self.map_height-y-1)
            for x in range(self.map_width):
                x_distance_from_edge = min(x, self.map_height-x-1)
                self.distance_from_edge[y,x] = y_distance_from_edge + x_distance_from_edge

        # relevant_set에 좌표가 들어오면, 그 좌표들 중 가장 가까운 거리 추출
        def calculate_distance_from_set(relevant_set):
            visited = set()
            # matrix 만듦
            matrix = self.init_matrix(default_value=-1)

            # x, y 좌표에는 0으로 표기
            for y in self.y_iteration_order:
                for x in self.x_iteration_order:
                    if (x,y) in relevant_set:
                        visited.add((x,y))
                        matrix[y,x] = 0

            # BFS
            queue = deque(list(visited))
            while queue:
                x,y = queue.popleft()
                for dx,dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                    xx, yy = x+dx, y+dy
                    if (xx,yy) in visited:
                        continue
                    if 0 <= xx < self.map_width and 0 <= yy < self.map_height:
                        matrix[yy,xx] = matrix[y,x] + 1
                        queue.append((xx,yy))
                        visited.add((xx,yy))
            return matrix

        # 각 set 마다 최소 distance를 구함
        self.distance_from_collectable_resource = calculate_distance_from_set(self.collectable_tiles_xy_set)
        self.distance_from_player_assets = calculate_distance_from_set(self.player_units_xy_set | self.player_city_tile_xy_set)
        self.distance_from_opponent_assets = calculate_distance_from_set(self.opponent_units_xy_set | self.opponent_city_tile_xy_set)
        self.distance_from_buildable_tile = calculate_distance_from_set(self.buildable_tile_xy_set)

        # 갈 수 있는 좌표를 모두 구함
        self.positions_to_calculate_distances_from = set()

        for unit in self.player.units:
            x,y = tuple(unit.pos)
            self.positions_to_calculate_distances_from.add((x,y),)
            if unit.can_act():
                self.positions_to_calculate_distances_from.add((x+1,y),)
                self.positions_to_calculate_distances_from.add((x-1,y),)
                self.positions_to_calculate_distances_from.add((x,y+1),)
                self.positions_to_calculate_distances_from.add((x,y-1),)

        # sy, sx로 부터 y,x 까지의 거리
        self.distance_matrix = np.full((self.map_height,self.map_width,self.map_height,self.map_width), 1001)


        for sy in range(self.map_height):
            for sx in range(self.map_width):
                # 만약 거리 계산을 할 필요가 없다면 continue
                if (sx,sy) not in self.positions_to_calculate_distances_from:
                    continue

                # 일단 default값 넣어두고
                blockade_multiplier_value_for_syx = blockade_multiplier_value
                
                # unit이 있는 공간이면 1
                if (sx,sy) in self.player_units_xy_set:
                    blockade_multiplier_value_for_syx = 1

                start_pos = (sx,sy)
                # 이미 방문한 위치 저장
                xy_processed = set()

                d4 = [(1,0),(0,1),(-1,0),(0,-1)]
                heap = [(0, start_pos),]
                while heap:
                    curdist, (x,y) = heapq.heappop(heap)
                    if (x,y) in xy_processed:
                        continue
                    xy_processed.add((x,y),)

                    # 도착했으면 distance 저장
                    self.distance_matrix[sy,sx,y,x] = curdist

                    # 4방향 BFS
                    for dx,dy in d4:
                        xx,yy = x+dx,y+dy
                        if not (0 <= xx < self.map_width and 0 <= yy < self.map_height):
                            continue
                        if (xx,yy) in xy_processed:
                            continue

                        edge_length = 1
                        # 만약 갈 수 없는 공간이면 default값 만큼 더하기
                        if (xx,yy) in self.occupied_xy_set:
                            edge_length = blockade_multiplier_value_for_syx
                        # 만약 city tile로 가려고 하면 default값 만큼 더하기
                        if (xx,yy) in self.player_city_tile_xy_set:
                            edge_length = blockade_multiplier_value_for_syx

                        heapq.heappush(heap, (curdist + edge_length, (xx,yy)))

    def retrieve_distance(self, sx, sy, ex, ey):
        return self.distance_matrix[sy,sx,ey,ex]


    # matrix가 주어지면 4방향 다 더하기
    def convolve(self, matrix):
        # each worker gets resources from (up to) five tiles
        new_matrix = matrix.copy()
        new_matrix[:-1,:] += matrix[1:,:]
        new_matrix[:,:-1] += matrix[:,1:]
        new_matrix[1:,:] += matrix[:-1,:]
        new_matrix[:,1:] += matrix[:,:-1]
        return new_matrix

    
    def calculate_resource_matrix(self):
        # 해당 tile에서 얻을 수 있는 resource의 개수
        self.collectable_tiles_matrix = self.wood_exist_matrix

        if self.player.researched_coal():
            self.collectable_tiles_matrix += self.coal_exist_matrix

        if self.player.researched_uranium():
            self.collectable_tiles_matrix += self.uranium_exist_matrix


        # 4방향 resource 더하기
        self.convolved_collectable_tiles_matrix = self.convolve(self.collectable_tiles_matrix)

        # resource matrix set으로 변환
        self.collectable_tiles_xy_set = set()  # exclude adjacent
        self.populate_set(self.collectable_tiles_matrix, self.collectable_tiles_xy_set)
        self.convolved_collectable_tiles_xy_set = set()  # include adjacent
        self.populate_set(self.convolved_collectable_tiles_matrix, self.convolved_collectable_tiles_xy_set)


    def calculate_resource_groups(self):
        # resource 들을 group으로 변환
        self.xy_to_resource_group_id: DisjointSet = DisjointSet()
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if (x,y) in self.collectable_tiles_xy_set:
                    # 만약 wood나 uranium이면 group의 point를 5로 설정
                    if (x,y) in self.wood_exist_xy_set or (x,y) in self.uranium_exist_xy_set:
                        self.xy_to_resource_group_id.find((x,y), point=5)
                    else:
                        self.xy_to_resource_group_id.find((x,y), point=1)

        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if (x,y) in self.collectable_tiles_xy_set:
                    # 주변 tile들을 하나의 group으로 합침
                    for dy,dx in [(1,0),(0,1),(-1,0),(0,-1)]:
                        xx, yy = x+dx, y+dy
                        if 0 <= yy < self.map_height and 0 <= xx < self.map_width:
                            self.xy_to_resource_group_id.union((x,y), (xx,yy))


    def repopulate_targets(self, missions: Missions):

        # target 들이 어디에 있는지 list
        pos_list = missions.get_targets()

        # target들이 속한 group을 추출
        self.targeted_leaders: Set = set(self.xy_to_resource_group_id.find(tuple(pos)) for pos in pos_list)
        # target들이 속한 group의 point를 셈
        self.targeted_cluster_count = sum(self.xy_to_resource_group_id.get_point((x,y)) > 0 for x,y in self.targeted_leaders)
        # target들의 좌표 중에서 city tile 좌표는 제외
        self.targeted_xy_set: Set = set(tuple(pos) for pos in pos_list) - self.player_city_tile_xy_set

        # 위치와 action 리스트
        pos_and_action_list = missions.get_targets_and_actions()

        # 집 짓는 위치 따로 추출
        self.targeted_for_building_xy_set: Set = \
            set(tuple(pos) for pos,action in pos_and_action_list if action and action[:5] == "bcity") - self.player_city_tile_xy_set


        # 유닛별로 돌면서 위치를 찾고, 해당 위치의 resource group leader들에 unit id를 모아둠
        self.resource_leader_to_locating_units: DefaultDict[Tuple, Set[str]] = defaultdict(set)
        for unit_id in self.player.units_by_id:
            unit: Unit = self.player.units_by_id[unit_id]
            current_position = tuple(unit.pos)
            leader = self.xy_to_resource_group_id.find(current_position)
            if leader:
                self.resource_leader_to_locating_units[leader].add(unit_id)

        # mission을 가진 unit들의 위치를 찾고, 해당 위치의 resource group leader들에게 unit id를 모아둠
        self.resource_leader_to_targeting_units: DefaultDict[Tuple, Set[str]] = defaultdict(set)
        for unit_id in missions:
            mission: Mission = missions[unit_id]
            target_position = tuple(mission.target_position)
            leader = self.xy_to_resource_group_id.find(target_position)
            if leader:
                self.resource_leader_to_targeting_units[leader].add(unit_id)
    


    def get_nearest_empty_tile_and_distance(self, current_position: Position, current_target: Position=None) -> Tuple[Position, int]:
        
        # 만약 현재 위치가 resource가 있는 장소라면
        if self.all_resource_amount_matrix[current_position.y, current_position.x] == 0:
            if tuple(current_position) not in self.player_city_tile_xy_set:
                return current_position, 0

        nearest_distance = 10**9+7
        nearest_position: Position = current_position

        # 모든 땅을 돌면서
        for y in self.y_iteration_order:
            for x in self.x_iteration_order:
                if (x,y) not in self.buildable_tile_xy_set:
                    continue

                # 다른 유닛이 지을려고하면 안짓고 다른데로 감
                if (x,y) in self.targeted_for_building_xy_set:
                    if current_target and (x,y) != tuple(current_target):
                        continue

                # 만약에 짓고 싶은 빈 땅이 resource 옆이 아니면 안 지음
                if self.distance_from_collectable_resource[y,x] != 1:
                    continue

                # 빈 땅 중에 가장 가까운 땅으로 감
                position = Position(x, y)
                distance = self.retrieve_distance(current_position.x, current_position.y, position.x, position.y)

                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_position = position

        return nearest_position, nearest_distance


In [5]:
# test heap

heap = [(0, (1,1)),]
heapq.heappush(heap, (-1, (1,2)))

heapq.heappop(heap)

(-1, (1, 2))